In [3]:
import pandas as pd
df_ml = pd.read_csv("../data/processed/ML_Chile_2022-12-27.csv")
df_f = pd.read_csv("../data/processed/FalabellaAyuda_2022-12-27.csv")
# df_ml = df_ml[df_ml["sentiment"] != "others"]
# df_f = df_f[df_f["sentiment"] != "others"]
# df_r = pd.read_csv("../data/processed/RipleyChile_2022-12-27.csv")
# df_p = pd.read_csv("../data/processed/tiendas_paris_2022-12-27.csv")
df = {
    "ML": df_ml,
    "Falabella": df_f,
}

In [4]:
# eliminar nan
for key, value in df.items():
    print(key)
    value.dropna(subset=["tweet_text"], inplace=True)

ML
Falabella


In [5]:
df_f.columns

Index(['user_id', 'user_location', 'created_at', 'tweet_text',
       '@usernames_in_tweet', 'hashtags_in_tweet', 'sentiment', 'score'],
      dtype='object')

In [6]:
# peridod en el que se encuentran los datos.
def periodo(df):
    # Encuentra la fecha de inicio (la primera fecha en el dataframe)
    fecha_inicio = df['created_at'].min()

    # Encuentra la fecha de finalización (la última fecha en el dataframe)
    fecha_finalizacion = df['created_at'].max()

    # Imprime la fecha de inicio y finalización
    print("Fecha de inicio:", fecha_inicio)
    print("Fecha de finalización:", fecha_finalizacion)
periodo(df_f)
periodo(df_ml)

Fecha de inicio: 2022-12-22 14:17:28+00:00
Fecha de finalización: 2022-12-27 15:31:58+00:00
Fecha de inicio: 2022-12-18 02:57:50+00:00
Fecha de finalización: 2022-12-27 14:31:29+00:00


In [7]:
# medición de la longitud de los microtextos.
import statistics as stats
resultado = {
    "tienda": [],
    "count": [],
    "mean": [],
    "std": [],
    "multimode": [],
    "max": [],
    "min": [],
    "quantiles": [],
}
text_length = []
for key, value in df.items():
    for row in value.iterrows():
        text_length.append(row[1][3].__len__())
    resultado["tienda"].append(key)
    resultado["count"].append(text_length.__len__())
    resultado["mean"].append(stats.mean(text_length))
    resultado["std"].append(stats.stdev(text_length))
    resultado["multimode"].append(stats.multimode(text_length))
    resultado["max"].append(max(text_length))
    resultado["min"].append(min(text_length))
    resultado["quantiles"].append(stats.quantiles(text_length))
    text_length = []
df_resultado = pd.DataFrame(resultado)
df_resultado

,tienda,count,mean,std,multimode,max,min,quantiles
0,ML,77,143.701299,76.886120,"[122, 130, 236, 139, 113, 114, 152, 73]",280,12,"[84.0, 139.0, 223.0]"
1,Falabella,434,139.421659,75.698042,[113],275,1,"[79.0, 126.5, 208.25]"


In [8]:
import nltk
nltk.download("stopwords")
from collections import Counter

def word_frequency(data):
    addi = ['si', 'mas', 'aun']
    # Descarga la lista de stop words del español
    words0 = []
    stop_words = nltk.corpus.stopwords.words("spanish")
    for i in addi:
        stop_words.append(i)
    # Inicializa un contador de palabras en cero
    word_count = Counter()
    
    # Itera a través de cada cadena en la lista de datos
    for text in data:
        # Separa la cadena en una lista de palabras
        words = text.split()
        
        # Elimina las stop words de la lista de palabras
        words = [word for word in words if word not in stop_words]
        for word in words:
            words0.append(word)
        
        # Actualiza el contador de palabras con la lista de palabras filtrada
    word_count.update(words0)
    return word_count

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/unknownsystem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from autocorrect import Speller
spell = Speller('es')
texts = []
resultado = {}
for key, value in df.items():
    for row in value.iterrows():
        texts.append(spell(row[1][3]))
    resultado[key] = word_frequency(texts)

In [10]:
keys = {}
for key, value in resultado.items():
    keys[key] = {}
    for top in sorted(value, key=value.get, reverse=True)[:5]:
        keys[key][top] = value[top]
keys

{'ML': {'necesito': 21, 'compra': 16, 'cuenta': 15, 'hoy': 14, 'ahora': 12},
 'Falabella': {'compra': 86,
  'necesito': 74,
  'regalo': 63,
  'producto': 62,
  'pedido': 54}}

In [11]:
traduccion = {
    "sadness": "tristeza",
    "joy": "alegría",
    "anger": "enojo",
    "surprise": "sorpresa",
    "disgust": "rechazo",
    "Fear": "miedo",
    "others": "otros"
}

In [12]:
# Analisis cuantitativo de los sentimientos.
from itertools import zip_longest
print(f"Mercado Libre & Falabella")
def sentimiento(df1, df2):
    count1 = dict(df1["sentiment"].value_counts())
    count2 = dict(df2["sentiment"].value_counts())
    for (key1, value1), (key2, value2) in zip_longest(count1.items(), count2.items(), fillvalue=("none", "none")):
        print(f"{traduccion[key1]} ({value1}) & {traduccion[key2]} ({value2})")

sentimiento(df_ml, df_f)

Mercado Libre & Falabella
otros (49) & otros (212)
enojo (16) & enojo (172)
tristeza (11) & tristeza (39)
alegría (1) & alegría (11)


In [13]:
# Análisis de hashtags
print(f"Mercado Libre & Falabella")
def hashtags(df1, df2):
    count1 = dict(df1["hashtags_in_tweet"].value_counts(dropna=False))
    count2 = dict(df2["hashtags_in_tweet"].value_counts(dropna=False))
    for (key1, value1), (key2, value2) in zip_longest(count1.items(), count2.items(), fillvalue=("none", "none")):
        print(f"{key1} ({value1}) & {key2} ({value2})")
hashtags(df_ml, df_f)

Mercado Libre & Falabella
nan (74) & nan (414)
['AlexMeyerFrankfurt', 'MercadoLibre', 'Cadem'] (1) & ['falabella'] (7)
['canal', 'Twitter', 'Chile', 'EngagementRate', 'datos', 'análisis'] (1) & ['pésimoservicio'] (1)
['ALANMAYERFRANKFURT'] (1) & ['Falabella'] (1)
none (none) & ['nocumplepromesacomercial'] (1)
none (none) & ['iquique'] (1)
none (none) & ['sernac'] (1)
none (none) & ['2036833842'] (1)
none (none) & ['WhatsApp'] (1)
none (none) & ['FalaferiaTeRoba'] (1)
none (none) & ['Incendio', 'VinadelMar'] (1)
none (none) & ['VINADELMAR', 'Incendiovinadelmar', 'Incendiovina'] (1)
none (none) & ['malservicio', 'rechazofalabella', 'sernac', 'Navidad2022'] (1)
none (none) & ['AyudaConCasos'] (1)
none (none) & ['falabella', 'negligentes'] (1)


In [14]:
# Analisis de usuarios. 'user_id', 'user_location'
print("Número de usuarios que publicaron n tweets & Mercado Libre & Falabella")
def users(df1, df2):
    vnr = {}
    vnr["Mercado Libre"] = df1['user_id'].value_counts()
    vnr["Falabella"] = df2['user_id'].value_counts()
    max_v = {
        "Mercado Libre": vnr["Mercado Libre"].max(),
        "Falabella": vnr["Falabella"].max(),
    }
    count = {}
    for tienda in vnr.keys():
        count[tienda] = {}
        for idx in range(1, max_v[tienda] + 1):
            count[tienda][f"Usuarios que publicaron {idx} tweets"] = len(vnr[tienda][vnr[tienda] == idx].keys())
    for (k1, v1), (k2, v2) in zip_longest(count["Mercado Libre"].items(), count["Falabella"].items(), fillvalue=("none", "none")):
        if k1 == k2 or k1 != "none":
            print(f"{k1} & {v1} & {v2}")
        else:
            print(f"{k2} & {v1} & {v2}")



users(df_ml, df_f)

Número de usuarios que publicaron n tweets & Mercado Libre & Falabella
Usuarios que publicaron 1 tweets & 49 & 198
Usuarios que publicaron 2 tweets & 11 & 53
Usuarios que publicaron 3 tweets & 2 & 13
Usuarios que publicaron 4 tweets & none & 4
Usuarios que publicaron 5 tweets & none & 6
Usuarios que publicaron 6 tweets & none & 5
Usuarios que publicaron 7 tweets & none & 1
Usuarios que publicaron 8 tweets & none & 1


In [16]:
print(f"Mercado Libre & Falabella")
def localizacion(df1, df2):
    count1 = dict(df1["user_location"].value_counts(dropna=False))
    count2 = dict(df2["user_location"].value_counts(dropna=False))
    for (key1, value1), (key2, value2) in zip_longest(count1.items(), count2.items(), fillvalue=("none", "none")):
        print(f"{key1} ({value1}) & {key2} ({value2})")
localizacion(df_ml, df_f)

Mercado Libre & Falabella
nan (35) & nan (206)
Chile (8) & Chile (40)
Santiago, Chile (7) & Santiago, Chile (34)
oriundo de San Miguel  (3) & Santiago de Chile (17)
chile (3) & Santiago (12)
Santiago - Chile (2) & Santiago, Metropolitana de Santiago (6)
Hogwarts  (2) & puente alto, la foresta (6)
Villa Alemana, Chile (2) & ÜT: -33.447159,-70.863025 (5)
Santiago  (2) & Valparaíso, Chile (5)
anywhere (1) & Concepción, Chile (5)
Chillán, Chile (1) & Maipú, Chile (3)
Antofagasta, CL (1) & chile (3)
Santiago (1) & Santiago,Chile (3)
Libertador San Martin (1) & Santiago / Chile (3)
chillan (1) & Concepcion, Chile (3)
Lejos (1) & Rancagua , Chile (3)
Maipú, Chile (1) & Las Condes, Chile (3)
Metropolitana de Santiago, Chi (1) & Concepción (2)
Coyhaique, Aysén del General C (1) & Talca (2)
Timboctu  (1) & Viña del Mar (2)
San Felipe, Chile (1) & Viña del Mar, Chile (2)
SANTIAGO (1) & Asteroide B612 (2)
none (none) & Paine, Chile (2)
none (none) & Santiago / Puerto Montt (2)
none (none) & Tel Av